In [ ]:
import networkx as nx

# Declare graphs

In [ ]:
G = nx.Graph() # for a directed graph use nx.DiGraph()
G.add_node(1)
G.add_nodes_from(range(2,9))  # add multiple nodes at once

# add edges 
G.add_edge(1,2)
edges = [(2,3), (1,3), (4,1), (4,5), (5,6), (5,7), (6,7), (7,8), (6,8)]
G.add_edges_from(edges)
G.nodes()
G.edges()
print(G)

### Pandas to nx

In [ ]:
# edges = pd.read_csv(data_folder + 'quakers_edgelist.csv')
# edges.columns -> "Source", "Target"

quakerG = nx.from_pandas_edgelist(edges, 'Source', 'Target', edge_attr=None, create_using= nx.Graph())
describe_graph(quakerG)

### Subgraph

In [ ]:
# Lets check by looking at the subgraphs induced by Alex and John
subgraph_Alex = quakerG.subgraph(['Alexander Parker']+list(quakerG.neighbors('Alexander Parker')))
subgraph_John = quakerG.subgraph(['John Crook']+list(quakerG.neighbors('John Crook')))

nx.draw_spring(subgraph_Alex, with_labels=True)

# Add atributes to nodes

In [ ]:
# add node attributes by passing dictionary of type name -> attribute
nx.set_node_attributes(quakerG, nodes['Role'].to_dict(), 'Role' )
nx.set_node_attributes(quakerG, nodes['Gender'].to_dict(), 'Gender' )
nx.set_node_attributes(quakerG, nodes['Birthdate'].to_dict(), 'Birthdate' )
nx.set_node_attributes(quakerG, nodes['Deathdate'].to_dict(), 'Deathdate' )
nx.set_node_attributes(quakerG, nodes['Quaker'].to_dict(), 'Quaker' )

quakerG.nodes['William Penn']
-> {
    'Role': 'Quaker leader and founder of Pennsylvania',
    'Gender': 'male',
    'Birthdate': 1644,
    'Deathdate': 1718,
    'Quaker': True
}

# Draw graph

In [ ]:
nx.draw_spring(G, with_labels=True,  alpha = 0.8)
nx.draw_circular(karateG, with_labels=True,  node_color='g', alpha = 0.8)

### Graph info

In [ ]:
# Helper function for plotting the degree distribution of a Graph
def plot_degree_distribution(G):
    degrees = {}
    for node in G.nodes():
        degree = G.degree(node)
        if degree not in degrees:
            degrees[degree] = 0
        degrees[degree] += 1
    sorted_degree = sorted(degrees.items())
    deg = [k for (k,v) in sorted_degree]
    cnt = [v for (k,v) in sorted_degree]
    fig, ax = plt.subplots()
    plt.bar(deg, cnt, width=0.80, color='b')
    plt.title("Degree Distribution")
    plt.ylabel("Frequency")
    plt.xlabel("Degree")
    ax.set_xticks([d+0.05 for d in deg])
    ax.set_xticklabels(deg)

In [ ]:
# Helper function for printing various graph properties
def describe_graph(G):
    print(G)
    if nx.is_connected(G):
        print("Avg. Shortest Path Length: %.4f" %nx.average_shortest_path_length(G))
        print("Diameter: %.4f" %nx.diameter(G)) # Longest shortest path
    else:
        print("Graph is not connected")
        print("Diameter and Avg shortest path length are not defined!")
    print("Sparsity: %.4f" %nx.density(G))  # #edges/#edges-complete-graph
    # #closed-triplets(3*#triangles)/#all-triplets
    print("Global clustering coefficient aka Transitivity: %.4f" %nx.transitivity(G))

In [ ]:
# Helper function for visualizing the graph
def visualize_graph(G, with_labels=True, k=None, alpha=1.0, node_shape='o'):
    #nx.draw_spring(G, with_labels=with_labels, alpha = alpha)
    pos = nx.spring_layout(G, k=k)
    if with_labels:
        lab = nx.draw_networkx_labels(G, pos, labels=dict([(n, n) for n in G.nodes()]))
    ec = nx.draw_networkx_edges(G, pos, alpha=alpha)
    nc = nx.draw_networkx_nodes(G, pos, nodelist=G.nodes(), node_color='g', node_shape=node_shape)
    plt.axis('off')

In [ ]:
describe_graph(quakerG)
visualize_graph(quakerG, False, k=0.2, alpha=0.4, node_shape='.')
plot_degree_distribution(karateG)


# Statistics

### Sparcity
"Sparsity" of a graph with $n$ nodes is defined as follows: 

$ L = \frac{|E|}{|E_{max}|}$, where $E_{max} = \frac{n * (n-1)}{2}$

In [ ]:
nx.density(quakerG)

### Connected Components

In [ ]:
print(nx.is_connected(quakerG))
comp = list(nx.connected_components(quakerG))
print('The graph contains', len(comp), 'connected components')

largest_comp = max(comp, key=len)
percentage_lcc = len(largest_comp)/quakerG.number_of_nodes() * 100
print('The largest component has', len(largest_comp), 'nodes', 'accounting for %.2f'% percentage_lcc, '% of the nodes') 

### Diameter and Shortest Paths

In [ ]:
fell_whitehead_path = nx.shortest_path(quakerG, source="Margaret Fell", target="George Whitehead")
print("Shortest path between Fell and Whitehead:", fell_whitehead_path)

# take the largest component and analyse its diameter = longest shortest-path
lcc_quakerG = quakerG.subgraph(largest_comp)
print("The diameter of the largest connected component is", nx.diameter(lcc_quakerG))
print("The avg shortest path length of the largest connected component is", nx.average_shortest_path_length(lcc_quakerG))

### Triadic Closure
A *friend* of my *friend* is my *friend*   
OR   
*quaker_1* knows *quaker_2* and *quaker_2* knows *quaker_3*, how likely is that *quaker_1* and *quaker_3* know each other?


In [ ]:
print('%.4f' %nx.transitivity(quakerG))

Employ a **local** measure called **clustering coefficient**, which quantifies for a node how close its neighbours are to being a clique (complete graph). Measured as the ratio of, the number of edges to the number of all possible edges, among the neighbors of a node.

In [ ]:
# Similar measure but for individual nodes called clustering coefficient
print(nx.clustering(quakerG, ['Alexander Parker', 'John Crook']))

### Importance

#### Degree

In [ ]:
### DEGREE
degrees = dict(quakerG.degree(quakerG.nodes()))
sorted_degree = sorted(degrees.items(), key=itemgetter(1), reverse=True)

# And the top 5 most popular quakers are.. 
for quaker, degree in sorted_degree[:5]:
    print(quaker, 'who is', quakerG.nodes[quaker]['Role'], 'knows', degree, 'people')
    
plot_degree_distribution(quakerG)


#### What about the Katz Centrality (the generalization over degree centrality)?

In [ ]:
degrees = dict(quakerG.degree(quakerG.nodes()))

katz = nx.katz_centrality(quakerG)
nx.set_node_attributes(quakerG, katz, 'katz')
sorted_katz = sorted(katz.items(), key=itemgetter(1), reverse=True)

# And the top 5 most popular quakers are.. 
for quaker, katzc in sorted_katz[:5]:
    print(quaker, 'who is', quakerG.nodes[quaker]['Role'], 'has katz-centrality: %.3f' %katzc)

#### Betweeness centrality: the more shortest paths pass through a node, the more important it is!

In [ ]:
# Compute betweenness centrality
betweenness = nx.betweenness_centrality(quakerG)
# Assign the computed centrality values as a node-attribute in your network
nx.set_node_attributes(quakerG, betweenness, 'betweenness')
sorted_betweenness = sorted(betweenness.items(), key=itemgetter(1), reverse=True)

for quaker, bw in sorted_betweenness[:5]:
    print(quaker, 'who is', quakerG.nodes[quaker]['Role'], 'has betweeness: %.3f' %bw)

### Homophily in quakers 
How likely is it that two quakers who have the same attribute are linked?

Try to measure the similarity of connections in the graph with respect to a given attribute.   
*Intuition: Like correlation, but translated to graphs.*

In [ ]:
print(nx.attribute_assortativity_coefficient(quakerG, 'Gender'))
print(nodes.groupby('Gender').size())

nx.numeric_assortativity_coefficient(quakerG, 'Deathdate')

# Communitie detection
Community detection is a common class of methods applied to graphs. 
Two important algorithms:
* **Girvan Newman**
* **Louvain**

### Girvan Newman
**Idea:** Edges possessing high betweeness centrality separate communities. Let's apply this on our toy sample graph (G) to get a better understanding of the idea.

In [ ]:
comp = girvan_newman(G)
it = 0
for communities in itertools.islice(comp, 4):
    it +=1
    print('Iteration', it)
    print(tuple(sorted(c) for c in communities)) 
    
visualize_graph(G,alpha=0.7)


### The [Louvain method](https://en.wikipedia.org/wiki/Louvain_Modularity)

Another clustering algorithm and has become a standard algorithm in the data scientist toolbox.   
**Idea:** It proceeds the other way around: initially every node is considered as a community. The communities are traversed, and for each community it is tested whether by joining it to a neighboring community, we can obtain a better clustering. 

In [ ]:
partition = community_louvain.best_partition(quakerG)
# add it as an attribute to the nodes
for n in quakerG.nodes:
    quakerG.nodes[n]["louvain"] = partition[n]
    
# plot it out
pos = nx.spring_layout(quakerG,k=0.2)
ec = nx.draw_networkx_edges(quakerG, pos, alpha=0.2)
nc = nx.draw_networkx_nodes(quakerG, pos, nodelist=quakerG.nodes(), node_color=[quakerG.nodes[n]["louvain"] for n in quakerG.nodes], 
                            node_size=100, cmap=plt.cm.jet)
plt.axis('off')
plt.show()

# Matching

### V1

In [ ]:
# TODO
def similarity_metric(test, control):
    dist = abs(test["Pred_Prob"] - control["Pred_Prob"])
    if dist > 0.05:
        return None
    else:
        return 1 - (dist)

G = nx.Graph()

# G.add_nodes_from(range(len(data_df)))
test_nodes = [f"test_{i}" for i in test_group.index]
control_nodes = [f"control_{i}" for i in control_group.index]

G.add_nodes_from(test_nodes, bipartite=0, group='test')
G.add_nodes_from(control_nodes, bipartite=1, group='control')

nx.set_node_attributes(G, data_df['Price'].to_dict(), 'Price' )
nx.set_node_attributes(G, data_df['Discounted_Price'].to_dict(), 'Discounted_Price' )
nx.set_node_attributes(G, data_df['Sold_within_3_months'].to_dict(), 'Sold_within_3_months' )       
# nx.draw_spring(G, with_labels=True,  alpha = 0.8)

In [ ]:
for idx_t, test in test_group.iterrows():
    for idx_c, control in control_group.iterrows():
        similarity = similarity_metric(test, control)
        if similarity is not None:
            G.add_edge(f"test_{idx_t}", f"control_{idx_c}", weight=similarity)

In [ ]:
nx.draw_spring(G, with_labels=True,  alpha = 0.8)


Getting the matching

In [ ]:
from networkx.algorithms.matching import max_weight_matching

# Compute the maximum weight matching
matching = max_weight_matching(G, maxcardinality=True)

# Print the matching
print(f"We have {len(matching)} successful matches!")
for u, v in matching:
    print(f"{u} - {v}, Weight: {G[u][v]['weight']}")

### V2

In [ ]:
treatment_df = data_df[data_df['Applied_Discount'] == 1]
control_df = data_df[data_df['Applied_Discount'] == 0]

print(f"There are {treatment_df.shape[0]} samples in the treated group.")
print(f"There are {control_df.shape[0]} samples in the control group.")

def get_similarity(propensity_score1, propensity_score2):
    '''Calculate similarity for instances with given propensity scores'''
    return 1 - np.abs(propensity_score1 - propensity_score2)

G = nx.Graph()

for treatment_id, treatment_row in treatment_df.iterrows():
    for control_id, control_row in control_df.iterrows():
        similarity = get_similarity(control_row['Pred_Prob'], treatment_row['Pred_Prob'])
        if similarity>0.95:
            G.add_weighted_edges_from([(control_id, treatment_id, similarity)])

matching = nx.max_weight_matching(G, maxcardinality=True)
print(f"We have {len(matching)} successful matches!")
for u, v in matching:
    print(f"{u} - {v}, Weight: {G[u][v]['weight']}")